In [21]:
import pandas as pd 
import string
from nltk.tokenize import word_tokenize 
from string import punctuation 
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import svm
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [7]:
with open('archivos/stopwords.txt') as archivo_stopword:
    lista_Stopword=[line.rstrip('\n') for line in archivo_stopword]

In [8]:
def Procesar_DatosML(xtrain,xtest):
   vectorizer = TfidfVectorizer(tokenizer=tokenize, max_df = 0.80, use_idf=True,min_df=1)
   #vectorizer    = CountVectorizer(min_df=1,binary=True) # metrica binaria
   vector_train  = vectorizer.fit_transform(xtrain)
   vector_test   = vectorizer.transform(xtest)
   #print(vectorizer.get_feature_names())
   return vector_train, vector_test, vectorizer

In [12]:
def tokenize(text):
        tokens = nltk.word_tokenize(text,  language='spanish')
        stems = []
        for item in tokens:
            stems.append(nltk.PorterStemmer().stem(item))
        return stems

In [28]:
def clean_corpus(corpus):
        xcorpus = corpus.get_values()
        for i in range(len(corpus)):
            xcorpus[i] = re.sub("[^a-zA-Z]", " ", corpus[i].lower())
            xcorpus[i] = ' '.join(xcorpus[i].split())
        return xcorpus

In [39]:

corpus = pd.read_csv('archivos/sinSWsinPOS.csv')
frases = corpus['A']
frases = clean_corpus(frases)
# 3. PARTIR CORPUS PARA ENTRENAR
xtrain, xtest, ytrain, ytest = tts(frases, corpus['B'], test_size=0.30)
# 4. TOKENIZACION + VECTORIZACIO 
xtrain, xtest, vectorizer = Procesar_DatosML(xtrain=xtrain, xtest=xtest)
Algoritmo = "SVM"
# 5. MODELO
if(Algoritmo=="SVM"):
    modelo = svm.SVC(kernel='linear') 
    modelo.fit(X=xtrain,y=ytrain)
if(Algoritmo=="Random Forest"):
    modelo=RandomForestClassifier(n_estimators=10)
    modelo.fit(X=xtrain,y=ytrain)
if(Algoritmo=="Naive Bayes"):
    modelo = GaussianNB()
    modelo.fit(X=xtrain.toarray() ,y=ytrain)
# 6. PREDICT + METRICAS
prediccion = modelo.predict(xtest)
##print(pd.crosstab(ytest, prediccion, rownames=['REAL'], colnames=['PREDICCION']))
Metricas="Algoritmo:"+Algoritmo + "\n\n"
Metricas=Metricas+classification_report(ytest, prediccion)
print("Accuracy:",metrics.accuracy_score(ytest, prediccion))

/home/manuel/anaconda3/envs/Sistemas_Recomendacion/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.to_numpy()' or '.array' instead.
  


Accuracy: 0.9138321995464853


In [40]:
xtrain

<1029x4786 sparse matrix of type '<class 'numpy.float64'>'
	with 19959 stored elements in Compressed Sparse Row format>

In [41]:
ytrain

512     NoAgresivo
1457      Agresivo
1116      Agresivo
585     NoAgresivo
212       Agresivo
           ...    
887     NoAgresivo
218       Agresivo
452     NoAgresivo
632     NoAgresivo
758       Agresivo
Name: B, Length: 1029, dtype: object